# 3 Player Tic Tac Game

3 LLM playing the Tic Tac Game

Prepare for joy!

Please note: this is built with openrouter api

In [25]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI

In [26]:
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

MODEL_GPT = 'openai/gpt-4'
MODEL_GEMINI = 'google/gemini-3-flash-preview'
MODEL_CLAUDE = 'anthropic/claude-sonnet-4.5'

OPEN_ROUTER_URL = 'https://openrouter.ai/api/v1'

if openrouter_api_key:
    print(f"OpenAI API Key exists and begins {openrouter_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-or-v1


In [27]:
openrouter = OpenAI(
    api_key=openrouter_api_key,
    base_url=OPEN_ROUTER_URL,
    default_headers={
        "HTTP-Referer": "http://localhost:8888",  # or your app/site URL
        "X-Title": "llm_engineering_course"       # optional label
    }
)

In [28]:
board = [[" " for _ in range(3)] for _ in range(3)]

players = [
    {"name": "GPT-4", "model": MODEL_GPT, "symbol": "X"},
    {"name": "Gemini", "model": MODEL_GEMINI, "symbol": "O"},
    {"name": "Claude", "model": MODEL_CLAUDE, "symbol": "△"},
]

current_player = 0

In [29]:
def print_board():
    for row in board:
        print("|".join(row))
        print("-"*5)

def check_winner(symbol):
    # rows
    for row in board:
        if all(cell == symbol for cell in row):
            return True

    # columns
    for col in range(3):
        if all(board[row][col] == symbol for row in range(3)):
            return True

    # diagonals
    if all(board[i][i] == symbol for i in range(3)):
        return True

    if all(board[i][2-i] == symbol for i in range(3)):
        return True

    return False

In [30]:
def get_ai_move(player):
    prompt = f"""
Board:
{board}

You are '{player['symbol']}'.

Return ONLY this JSON format:
{{"row": 0, "col": 2}}

Do not explain.
Do not think out loud.
Do not include text.
Only JSON.
"""

    response = openrouter.chat.completions.create(
        model=player["model"],
        messages=[{
            "role": "system",
            "content": "You are a Tic Tac Toe AI. You must ONLY return valid JSON."
        },
        {
            "role": "user", 
            "content": prompt
        }
    ]
    )

    content = response.choices[0].message.content.strip()

    # Extract JSON safely
    start = content.find("{")
    end = content.rfind("}") + 1
    json_string = content[start:end]

    data = json.loads(json_string)

    row = data["row"]
    col = data["col"]

    return row, col

In [31]:
while True:
    player = players[current_player]
    print(f"\n{player['name']} ({player['symbol']}) turn")

    try:
        row, col = get_ai_move(player)

        if board[row][col] != " ":
            print("Invalid move. Skipping.")
        else:
            board[row][col] = player["symbol"]

            print_board()

            if check_winner(player["symbol"]):
                print(f"{player['name']} wins!")
                break

            if all(cell != " " for row in board for cell in row):
                print("Draw!")
                break

            current_player = (current_player + 1) % 3

    except Exception as e:
        print("Error:", e)
        break


GPT-4 (X) turn
 | |X
-----
 | | 
-----
 | | 
-----

Gemini (O) turn
 | |X
-----
 |O| 
-----
 | | 
-----

Claude (△) turn
△| |X
-----
 |O| 
-----
 | | 
-----

GPT-4 (X) turn
△| |X
-----
X|O| 
-----
 | | 
-----

Gemini (O) turn
△|O|X
-----
X|O| 
-----
 | | 
-----

Claude (△) turn
△|O|X
-----
X|O| 
-----
△| | 
-----

GPT-4 (X) turn
△|O|X
-----
X|O| 
-----
△|X| 
-----

Gemini (O) turn
△|O|X
-----
X|O| 
-----
△|X|O
-----

Claude (△) turn
△|O|X
-----
X|O|△
-----
△|X|O
-----
Draw!
